# EMA(20,50,200) + ADX Backtest Notebook — Patched

Patched version: uses custom indicator implementations (no `pandas_ta`) and safe signal generation. Fully parameterized — edit the **Parameters** cell and run top-to-bottom.

**Changes in this patched notebook:**
- Custom EMA/ATR/ADX implementations using pandas/numpy (Wilder smoothing for ATR and ADX).
- Defensive handling of NaNs and early rows; notebook drops initial rows where indicators are incomplete.
- `backtest_universe_custom()` runs the patched workflow.

---

Run the setup cell to install dependencies, then run all cells.


In [37]:
# ----------------------------
# PARAMETERS (edit these)
# ----------------------------
STOCKS = ['SBICARD.NS', 'BDL.NS', 'INDHOTEL.NS', 'BSE.NS', 'NYKAA.NS', 'BAJFINANCE.NS', 'PAYTM.NS', 'SOLARINDS.NS', 'CHOLAFIN.NS', 'UNITDSPR.NS', 'DIVISLAB.NS', 'MUTHOOTFIN.NS', 'BHARTIARTL.NS', 'ICICIBANK.NS', 'MAZDOCK.NS', 'SHREECEM.NS', 'DIXON.NS', 'PERSISTENT.NS', 'SRF.NS', 'TVSMOTOR.NS', 'SBILIFE.NS', 'MAXHEALTH.NS', 'MFSL.NS', 'COFORGE.NS', 'HDFCLIFE.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'HDFCBANK.NS', 'BEL.NS', 'BAJAJFINSV.NS']

START_DATE = '2018-01-01'
END_DATE = '2025-09-19'                # change as needed
TIMEFRAME = '1d'                       # yfinance uses '1d' for daily. (This notebook expects daily bars)
ADX_THRESHOLD = 25                     # ADX filter
ATR_LENGTH = 14
ATR_MULTIPLIER = 1.0                   # stop = entry - ATR*mult for long (and + for short)
RISK_PER_TRADE = 0.01                  # fraction of equity to risk per trade (1% default)
INITIAL_CAPITAL = 100000.0             # starting capital in INR (or chosen currency)
MAX_HOLD_DAYS = 10                     # optional: force exit after N trading days
TAKE_PROFIT_RR = 2.0                   # optional fixed target in R:R (e.g., 2.0 means 2x risk)
USE_SHORTS = True                      # enable short trades
BROKERAGE_PER_TRADE = 20.0             # flat brokerage per trade (INR) – adjust for your broker
SLIPPAGE_PCT = 0.0005                  # 0.05% slippage
PRINT_PROGRESS = True                  # show progress while backtesting
# ----------------------------


In [38]:
# Install required libraries (run this cell once)
# !pip install yfinance matplotlib pandas numpy --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import math


In [39]:
# Robust data download helper (normalizes column names)
def download_stock(ticker, start, end, interval='1d', threads=True):
    df = yf.download(ticker, start=start, end=end, interval=interval, progress=False, multi_level_index=False, auto_adjust=True)
    if df is None or df.empty:
        raise ValueError(f'No data for {ticker} from {start} to {end}')
    # Normalize columns
    col_map = {}
    cols = list(df.columns)
    # map common variants
    for c in cols:
        lc = c.lower()
        if 'open' in lc:
            col_map[c] = 'Open'
        elif 'high' in lc:
            col_map[c] = 'High'
        elif 'low' in lc:
            col_map[c] = 'Low'
        elif 'close' in lc and 'adj' not in lc:
            col_map[c] = 'Close'
        elif 'adj' in lc and 'close' in lc:
            col_map[c] = 'Adj_Close'
        elif 'volume' in lc:
            col_map[c] = 'Volume'
    df = df.rename(columns=col_map)
    # fallbacks
    if 'Adj_Close' not in df.columns:
        if 'Close' in df.columns:
            df['Adj_Close'] = df['Close']
        else:
            raise KeyError("Downloaded data missing Close/Adj Close columns.")
    required = ['Open','High','Low','Close','Adj_Close','Volume']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns after normalization: {missing}. Found: {df.columns.tolist()}")
    df = df[['Open','High','Low','Close','Adj_Close','Volume']].copy()
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    return df

# quick check (safe)
print('Sanity check: downloading first ticker...')
_sample = download_stock(STOCKS[0], START_DATE, END_DATE, TIMEFRAME)
print('Downloaded rows:', len(_sample))
_sample.head()


Sanity check: downloading first ticker...
Downloaded rows: 1367


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2020-03-16,651.218447,743.827424,646.292437,671.316589,671.316589,60887005
2020-03-17,680.774468,736.881725,678.804064,720.084045,720.084045,15145135
2020-03-18,740.970323,757.620259,669.937290,679.001160,679.001160,8949832
2020-03-19,650.233309,706.389823,640.381290,681.809021,681.809021,6432046
2020-03-20,695.798814,719.197358,680.035584,713.778748,713.778748,6492226


In [40]:
# ---------------------------
# Custom indicator implementations (no pandas_ta)
# ---------------------------
def ema(series, length):
    return series.ewm(span=length, adjust=False).mean()

def true_range(df):
    high = df['High']
    low = df['Low']
    prev_close = df['Close'].shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr

def atr(df, length=14):
    tr = true_range(df)
    return tr.ewm(alpha=1/length, adjust=False).mean()

def directional_movements(df):
    up_move = df['High'] - df['High'].shift(1)
    down_move = df['Low'].shift(1) - df['Low']
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    plus_dm = pd.Series(plus_dm, index=df.index)
    minus_dm = pd.Series(minus_dm, index=df.index)
    return plus_dm, minus_dm

def adx(df, length=14):
    tr = true_range(df)
    atr_series = tr.ewm(alpha=1/length, adjust=False).mean()
    plus_dm, minus_dm = directional_movements(df)
    plus_dm_smooth = plus_dm.ewm(alpha=1/length, adjust=False).mean()
    minus_dm_smooth = minus_dm.ewm(alpha=1/length, adjust=False).mean()
    di_plus = 100 * (plus_dm_smooth / atr_series).replace([np.inf, -np.inf], 0).fillna(0)
    di_minus = 100 * (minus_dm_smooth / atr_series).replace([np.inf, -np.inf], 0).fillna(0)
    dx = ( (di_plus - di_minus).abs() / (di_plus + di_minus).replace(0, np.nan) ) * 100
    dx = dx.replace([np.inf, -np.inf], np.nan).fillna(0)
    adx_series = dx.ewm(alpha=1/length, adjust=False).mean()
    out = pd.DataFrame({'ADX': adx_series, 'DI+': di_plus, 'DI-': di_minus}, index=df.index)
    return out

def add_indicators_custom(df):
    df = df.copy()
    df['EMA20'] = ema(df['Close'], 20)
    df['EMA50'] = ema(df['Close'], 50)
    df['EMA200'] = ema(df['Close'], 200)
    df['ATR'] = atr(df, ATR_LENGTH)
    adx_df = adx(df, 14)
    df['ADX'] = adx_df['ADX']
    df['DI+'] = adx_df['DI+']
    df['DI-'] = adx_df['DI-']
    return df

# test
_sample = add_indicators_custom(_sample)
_sample[['Close','EMA20','EMA50','EMA200','ATR','ADX']].tail()


,Close,EMA20,EMA50,EMA200,ATR,ADX
Date,,,,,,
2025-09-12,856.150024,821.747695,840.323270,838.480289,18.654827,23.028037
2025-09-15,900.049988,829.205057,842.665495,839.092923,21.283055,24.988881
2025-09-16,896.549988,835.618860,844.778612,839.664635,20.769977,26.880318
2025-09-17,893.299988,841.112300,846.681411,840.198320,20.357836,28.564643
2025-09-18,891.700012,845.930178,848.446846,840.710775,20.175133,29.236415


In [41]:
# Safe signal generation
def generate_signals_safe(df):
    df = df.copy()
    df['EMA20_gt_EMA50'] = (df['EMA20'] > df['EMA50']).fillna(False)
    df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)
    df['long_cross'] = (df['EMA20_gt_EMA50'] == True) & (df['EMA20_gt_EMA50_prev'] == False)
    df['short_cross'] = (df['EMA20_gt_EMA50'] == False) & (df['EMA20_gt_EMA50_prev'] == True)
    df['price_above_200'] = (df['Close'] > df['EMA200']).fillna(False)
    df['price_below_200'] = (df['Close'] < df['EMA200']).fillna(False)
    df['adx_ok'] = (df['ADX'] > ADX_THRESHOLD).fillna(False)
    df['signal_long'] = df['long_cross'] & df['price_above_200'] & df['adx_ok']
    df['signal_short'] = df['short_cross'] & df['price_below_200'] & df['adx_ok']
    return df

# test
_sample2 = generate_signals_safe(_sample)
_sample2[['Close','EMA20','EMA50','EMA200','ADX','signal_long','signal_short']].tail(10)


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_85415/3657023606.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


,Close,EMA20,EMA50,EMA200,ADX,signal_long,signal_short
Date,,,,,,,
2025-09-05,791.450012,809.759352,840.704544,838.516329,22.571011,False,False
2025-09-08,803.849976,809.196554,839.259267,838.171390,21.801190,False,False
2025-09-09,819.349976,810.163547,838.478511,837.984112,20.327207,False,False
2025-09-10,854.400024,814.376545,839.102884,838.147455,21.014780,False,False
2025-09-11,853.750000,818.126398,839.677280,838.302704,22.116120,False,False
2025-09-12,856.150024,821.747695,840.323270,838.480289,23.028037,False,False
2025-09-15,900.049988,829.205057,842.665495,839.092923,24.988881,False,False
2025-09-16,896.549988,835.618860,844.778612,839.664635,26.880318,False,False
2025-09-17,893.299988,841.112300,846.681411,840.198320,28.564643,False,False


In [42]:
# Backtester (fills at next open) - adapted from previous implementation
from collections import namedtuple
Trade = namedtuple('Trade', ['ticker','entry_date','entry_price','side','size','stop','take_profit','exit_date','exit_price','pnl'])

def run_backtest_for_symbol(df, initial_capital=INITIAL_CAPITAL):
    equity = initial_capital
    cash = initial_capital
    position = None
    trades = []
    equity_curve = []
    peak_equity = initial_capital
    drawdowns = []

    for i in range(1, len(df)-1):
        row = df.iloc[i]
        next_row = df.iloc[i+1]

        if position is None:
            if row.get('signal_long', False):
                entry_price = next_row['Open'] * (1 + SLIPPAGE_PCT)
                atr = row['ATR'] if not np.isnan(row['ATR']) else 0
                stop = entry_price - ATR_MULTIPLIER * atr
                if stop >= entry_price: continue
                risk_amount = equity * RISK_PER_TRADE
                risk_per_share = entry_price - stop
                if risk_per_share <= 0: continue
                size_shares = math.floor(risk_amount / risk_per_share)
                if size_shares <= 0: continue
                position = {'side':'long','entry_price':entry_price,'size':size_shares,'stop':stop,
                            'take_profit': entry_price + TAKE_PROFIT_RR * risk_per_share if TAKE_PROFIT_RR else None,
                            'entry_date':next_row.name}
                cash -= entry_price * size_shares + BROKERAGE_PER_TRADE
            elif USE_SHORTS and row.get('signal_short', False):
                entry_price = next_row['Open'] * (1 - SLIPPAGE_PCT)
                atr = row['ATR'] if not np.isnan(row['ATR']) else 0
                stop = entry_price + ATR_MULTIPLIER * atr
                risk_amount = equity * RISK_PER_TRADE
                risk_per_share = stop - entry_price
                if risk_per_share <= 0: continue
                size_shares = math.floor(risk_amount / risk_per_share)
                if size_shares <= 0: continue
                position = {'side':'short','entry_price':entry_price,'size':size_shares,'stop':stop,
                            'take_profit': entry_price - TAKE_PROFIT_RR * risk_per_share if TAKE_PROFIT_RR else None,
                            'entry_date':next_row.name}
                cash -= BROKERAGE_PER_TRADE
        else:
            exit_flag = False
            exit_price = None

            if position['side'] == 'long':
                if row['Low'] <= position['stop']:
                    exit_flag = True
                    exit_price = position['stop'] * (1 - SLIPPAGE_PCT)
                elif position.get('take_profit') and row['High'] >= position['take_profit']:
                    exit_flag = True
                    exit_price = position['take_profit'] * (1 - SLIPPAGE_PCT)
                elif row['EMA20'] < row['EMA50'] or row['Close'] < row['EMA50']:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 - SLIPPAGE_PCT)
            else:
                if row['High'] >= position['stop']:
                    exit_flag = True
                    exit_price = position['stop'] * (1 + SLIPPAGE_PCT)
                elif position.get('take_profit') and row['Low'] <= position['take_profit']:
                    exit_flag = True
                    exit_price = position['take_profit'] * (1 + SLIPPAGE_PCT)
                elif row['EMA20'] > row['EMA50'] or row['Close'] > row['EMA50']:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 + SLIPPAGE_PCT)

            if not exit_flag and MAX_HOLD_DAYS:
                held_days = (row.name - position['entry_date']).days if isinstance(row.name, pd.Timestamp) else 0
                if held_days >= MAX_HOLD_DAYS:
                    exit_flag = True
                    exit_price = next_row['Open'] * (1 - SLIPPAGE_PCT if position['side']=='long' else 1 + SLIPPAGE_PCT)

            if exit_flag:
                if position['side'] == 'long':
                    exit_val = exit_price * position['size']
                    entry_val = position['entry_price'] * position['size']
                    pnl = exit_val - entry_val - BROKERAGE_PER_TRADE
                    cash += exit_val - BROKERAGE_PER_TRADE
                else:
                    entry_val = position['entry_price'] * position['size']
                    exit_val = exit_price * position['size']
                    pnl = entry_val - exit_val - BROKERAGE_PER_TRADE
                    cash += (entry_val - exit_val) - BROKERAGE_PER_TRADE

                equity = cash
                trades.append(Trade(ticker=None, entry_date=position['entry_date'], entry_price=position['entry_price'],
                                    side=position['side'], size=position['size'], stop=position['stop'],
                                    take_profit=position.get('take_profit'), exit_date=row.name, exit_price=exit_price, pnl=pnl))
                position = None

        if position is None:
            mtm = cash
        else:
            if position['side'] == 'long':
                mtm = cash + row['Close'] * position['size']
            else:
                mtm = cash + (position['entry_price'] - row['Close']) * position['size']
        equity_curve.append({'date':row.name, 'equity':mtm})
        if mtm > peak_equity:
            peak_equity = mtm
        drawdowns.append((peak_equity - mtm) / peak_equity if peak_equity>0 else 0)

    eq_df = pd.DataFrame(equity_curve).set_index('date')
    trades_df = pd.DataFrame(trades)
    return trades_df, eq_df, (max(drawdowns)*100 if drawdowns else 0)

def backtest_universe_custom(tickers):
    all_trades = []
    equity_tracks = {}
    max_dd_per_symbol = {}
    for t in tickers:
        if PRINT_PROGRESS:
            print('Running:', t)
        df = download_stock(t, START_DATE, END_DATE, TIMEFRAME)
        df = add_indicators_custom(df)
        df = df.dropna(subset=['EMA20','EMA50','EMA200','ADX','ATR']).copy()
        if df.empty:
            if PRINT_PROGRESS:
                print(f"Not enough data/indicators for {t}; skipping.")
            continue
        df = generate_signals_safe(df)
        trades_df, eq_df, max_dd = run_backtest_for_symbol(df, initial_capital=INITIAL_CAPITAL)
        trades_df['ticker'] = t
        all_trades.append(trades_df)
        equity_tracks[t] = eq_df
        max_dd_per_symbol[t] = max_dd
    all_trades_df = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame()
    return all_trades_df, equity_tracks, max_dd_per_symbol


In [43]:
# Run the backtest for the defined STOCKS universe.
# WARNING: This will download data and run the backtest for each ticker.
# Uncomment and run when ready.

trades, equity_tracks, max_dds = backtest_universe_custom(STOCKS)
print('Max drawdowns per symbol (%)', max_dds)
display(trades.head())
print('Backtest cell ready. Uncomment the run lines to execute the backtest.')

Running: SBICARD.NS
Running: BDL.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_85415/3657023606.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_85415/3657023606.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


Running: INDHOTEL.NS


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_85415/3657023606.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


Running: BSE.NS


...

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_85415/3657023606.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


Running: BAJAJFINSV.NS
Max drawdowns per symbol (%) {'SBICARD.NS': np.float64(1.0544212492487568), 'BDL.NS': np.float64(3.8962604330462636), 'INDHOTEL.NS': np.float64(2.437119580752478), 'BSE.NS': np.float64(3.3287045890809117), 'NYKAA.NS': np.float64(1.2321611659105483), 'BAJFINANCE.NS': np.float64(5.891318978179407), 'PAYTM.NS': np.float64(0.4176590234375035), 'SOLARINDS.NS': np.float64(3.370612137570477), 'CHOLAFIN.NS': np.float64(3.3808051201818743), 'UNITDSPR.NS': 0.0, 'DIVISLAB.NS': np.float64(3.7593923913487224), 'MUTHOOTFIN.NS': np.float64(2.4467989207080083), 'BHARTIARTL.NS': np.float64(2.087519618764389), 'ICICIBANK.NS': np.float64(0.09079281438347243), 'MAZDOCK.NS': np.float64(0.738061018646395), 'SHREECEM.NS': np.float64(3.2998014775784985), 'DIXON.NS': np.float64(1.2179070945290849), 'PERSISTENT.NS': 0.0, 'SRF.NS': np.float64(3.50023992492533), 'TVSMOTOR.NS': np.float64(2.2671531839629457), 'SBILIFE.NS': np.float64(1.5852080521027065), 'MAXHEALTH.NS': np.float64(1.57865332

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_85415/3657023606.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EMA20_gt_EMA50_prev'] = df['EMA20_gt_EMA50'].shift(1).fillna(False)


,ticker,entry_date,entry_price,side,size,stop,take_profit,exit_date,exit_price,pnl
0,SBICARD.NS,2023-07-26,878.524828,long,55.0,860.511061,914.552362,2023-07-26,860.080805,-1034.421249
1,BDL.NS,2018-06-04,176.317143,long,204.0,171.419235,186.112959,2018-06-04,171.333525,-1036.658021
2,BDL.NS,2018-06-07,168.295438,short,209.0,173.010971,158.864373,2018-06-07,173.097477,-1023.625990
3,BDL.NS,2018-06-25,166.416044,short,241.0,170.463445,158.321241,2018-07-05,162.010546,1041.724942
4,BDL.NS,2019-12-16,133.591650,short,164.0,139.603700,121.567550,2019-12-26,139.673501,-1017.423697


Backtest cell ready. Uncomment the run lines to execute the backtest.


In [44]:
# Performance summary helpers
def summary_from_trades(trades_df, equity_df, initial_capital=INITIAL_CAPITAL):
    total_trades = len(trades_df)
    wins = trades_df[trades_df['pnl']>0]
    losses = trades_df[trades_df['pnl']<=0]
    win_rate = len(wins)/total_trades if total_trades>0 else np.nan
    total_pnl = trades_df['pnl'].sum() if total_trades>0 else 0.0
    avg_win = wins['pnl'].mean() if len(wins)>0 else 0.0
    avg_loss = losses['pnl'].mean() if len(losses)>0 else 0.0
    final_equity = equity_df['equity'].iloc[-1] if not equity_df.empty else initial_capital
    total_return = (final_equity - initial_capital)/initial_capital * 100
    rolling_max = equity_df['equity'].cummax()
    drawdown = (rolling_max - equity_df['equity']) / rolling_max
    max_dd = drawdown.max() * 100 if not drawdown.empty else 0.0
    return {'total_trades':total_trades,'win_rate':win_rate,'total_pnl':total_pnl,'avg_win':avg_win,'avg_loss':avg_loss,'final_equity':final_equity,'total_return_pct':total_return,'max_dd_pct':max_dd}


In [45]:
# Plot equity curve for a symbol (after running backtest)
def plot_equity(equity_df, title='Equity Curve'):
    plt.figure(figsize=(12,5))
    plt.plot(equity_df.index, equity_df['equity'])
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Equity')
    plt.grid(True)
    plt.show()


## Notes & Next steps

- This patched notebook replaces pandas_ta with custom indicator implementations using pandas/numpy.
- It drops early rows missing indicators and uses safe boolean handling to avoid TypeError caused by NaNs.
- For portfolio-level capital allocation across multiple simultaneous positions, further changes are needed.
- Consider connecting to a reliable data vendor for production testing; yfinance is fine for prototyping.
